**Importações das bibliotecas.**

In [4]:
# Importando as bibliotecas necessarias

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
import datetime as dt
%matplotlib inline
# from google.colab import drive
# drive.mount('/content/drive')

**1. Introdução e Coleta dos Dados**


In [14]:
pathFile = "content/semanal-municipios-2022-2024.csv" # vscode 
#pathFile = "/content/semanal-municipios-2022-2024.csv"# collab
data = pd.read_csv(pathFile, sep=";")


In [15]:
data.columns

Index(['DATA INICIAL', 'DATA FINAL', 'REGIÃO', 'ESTADO', 'MUNICÍPIO',
       'PRODUTO', 'NÚMERO DE POSTOS PESQUISADOS', 'UNIDADE DE MEDIDA',
       'PREÇO MÉDIO REVENDA', 'DESVIO PADRÃO REVENDA', 'PREÇO MÍNIMO REVENDA',
       'PREÇO MÁXIMO REVENDA', 'MARGEM MÉDIA REVENDA',
       'COEF DE VARIAÇÃO REVENDA'],
      dtype='object')

**Renomeando colunas**

Como os nomes das colunas do dataset não estão na forma mais conveniente, isto é, sem espaços ou acentos, vamos, primeiramente, ajustá-los

In [17]:
def get_string_adjuster():
    untranslated_string = ' ÁÃÇÍÉÊÚ'
    translated_string = '_AACIEEU'

    def adjust_string(to_adjust):
        translation_map = to_adjust.maketrans(untranslated_string, translated_string)
        return to_adjust.translate(translation_map)

    return adjust_string

In [18]:
adjust_string = get_string_adjuster()

column_names_to_adjusted_names = {
    original_name : adjust_string(original_name)
    for original_name
    in data.columns
}

data = data.rename(columns = column_names_to_adjusted_names)
data.columns

Index(['DATA_INICIAL', 'DATA_FINAL', 'REGIAO', 'ESTADO', 'MUNICIPIO',
       'PRODUTO', 'NUMERO_DE_POSTOS_PESQUISADOS', 'UNIDADE_DE_MEDIDA',
       'PRECO_MEDIO_REVENDA', 'DESVIO_PADRAO_REVENDA', 'PRECO_MINIMO_REVENDA',
       'PRECO_MAXIMO_REVENDA', 'MARGEM_MEDIA_REVENDA',
       'COEF_DE_VARIACAO_REVENDA'],
      dtype='object')

**Tipo de cada coluna:**

Objetivamos definir o tipo de cada coluna e, caso necessário, ajustá-lo para um condizente com o mostrado

In [19]:
data.dtypes

DATA_INICIAL                    object
DATA_FINAL                      object
REGIAO                          object
ESTADO                          object
MUNICIPIO                       object
PRODUTO                         object
NUMERO_DE_POSTOS_PESQUISADOS     int64
UNIDADE_DE_MEDIDA               object
PRECO_MEDIO_REVENDA             object
DESVIO_PADRAO_REVENDA           object
PRECO_MINIMO_REVENDA            object
PRECO_MAXIMO_REVENDA            object
MARGEM_MEDIA_REVENDA            object
COEF_DE_VARIACAO_REVENDA        object
dtype: object

In [20]:
data.head()

,DATA_INICIAL,DATA_FINAL,REGIAO,ESTADO,MUNICIPIO,PRODUTO,NUMERO_DE_POSTOS_PESQUISADOS,UNIDADE_DE_MEDIDA,PRECO_MEDIO_REVENDA,DESVIO_PADRAO_REVENDA,PRECO_MINIMO_REVENDA,PRECO_MAXIMO_REVENDA,MARGEM_MEDIA_REVENDA,COEF_DE_VARIACAO_REVENDA
0,2/1/2022,8/1/2022,SUDESTE,SAO PAULO,ADAMANTINA,ETANOL HIDRATADO,7,R$/l,"4,656","0,078","4,580","4,790",-,"0,017"
1,2/1/2022,8/1/2022,CENTRO OESTE,GOIAS,AGUAS LINDAS DE GOIAS,ETANOL HIDRATADO,4,R$/l,"5,199","0,001","5,198","5,199",-,"0,000"
2,2/1/2022,8/1/2022,SUL,RIO GRANDE DO SUL,ALEGRETE,ETANOL HIDRATADO,2,R$/l,"6,845","0,643","6,390","7,299",-,"0,094"
3,2/1/2022,8/1/2022,NORTE,PARA,ALENQUER,ETANOL HIDRATADO,1,R$/l,"7,120","0,000","7,120","7,120",-,"0,000"
4,2/1/2022,8/1/2022,SUDESTE,MINAS GERAIS,ALFENAS,ETANOL HIDRATADO,2,R$/l,"5,499","0,000","5,499","5,499",-,"0,000"


Vamos começar convertendo os tipos das datas para objetos datetime de fato utilizando pd.to_datetime

In [21]:
data['DATA_INICIAL'] = pd.to_datetime(data['DATA_INICIAL'] ,format="%d/%m/%Y")
data['DATA_FINAL'] = pd.to_datetime(data['DATA_FINAL'],format="%d/%m/%Y")
data[['DATA_INICIAL', 'DATA_FINAL']].dtypes

DATA_INICIAL    datetime64[ns]
DATA_FINAL      datetime64[ns]
dtype: object

Consideramos que região e estado devem ser categóricos

In [22]:
data['REGIAO'] = data['REGIAO'].astype('category')
data['ESTADO'] = data['ESTADO'].astype('category')
data[['REGIAO', 'ESTADO']].dtypes

REGIAO    category
ESTADO    category
dtype: object

In [23]:
data['PRODUTO'].unique()

array(['ETANOL HIDRATADO', 'GASOLINA ADITIVADA', 'GASOLINA COMUM', 'GLP',
       'GNV', 'OLEO DIESEL', 'OLEO DIESEL S10'], dtype=object)

In [24]:
data['PRODUTO'] = data['PRODUTO'].astype('category')

data['UNIDADE_DE_MEDIDA'].unique()

array(['R$/l', 'R$/13kg', 'R$/m³'], dtype=object)

Há três unidades de medida diferentes para preços pesquisados. Poderíamos descobrir a proporção de litros para cada, de forma a ter o valor do litro para cada linha. Porém, os valores de desvio padrão e coeficiente de variância já estão calculados nas medidas originais, nos impossibilitando de obtê-los para as possíveis novas medidas. Sendo assim, vamos considerar três datasets - um para cada unidade de medida.

In [33]:
print({
    unit : f"{100 * data[data['UNIDADE_DE_MEDIDA'] == unit]['UNIDADE_DE_MEDIDA'].count() / data['UNIDADE_DE_MEDIDA'].count()}%"
    for unit
    in data['UNIDADE_DE_MEDIDA'].unique()
})

{'R$/l': '79.88177339901478%', 'R$/13kg': '15.842364532019705%', 'R$/m³': '4.275862068965517%', nan: '0.0%'}


Como observado, a unidade de medida menos recorrente constitui mais de 10% do dataset, de forma que nenhuma delas é desprezível. Dessa forma, realizaremos análises para cada das unidades de medidas disponíveis.


---


Antes de fazermos isso, vamos continuar convertendo valores para seus tipos apropriados.

In [25]:
data['MARGEM_MEDIA_REVENDA'].describe()

count     313500
unique         1
top            -
freq      313500
Name: MARGEM_MEDIA_REVENDA, dtype: object

Há valores em MARGEM_MEDIA_REVENDA que não são NaN, mas são hífens. Vamos converter esses valores para NaNs e atribuir o tipo float64 à coluna.

In [27]:
data['MARGEM_MEDIA_REVENDA'] = data['MARGEM_MEDIA_REVENDA'].replace({ '-': np.nan })
data['MARGEM_MEDIA_REVENDA'] = data['MARGEM_MEDIA_REVENDA'].astype('float64')
data['MARGEM_MEDIA_REVENDA'].describe()

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: MARGEM_MEDIA_REVENDA, dtype: float64